# Demo Recorder — Interactive Notebook

This notebook lets you run and experiment with the `demo-recorder` using `browser-use`.

**Prerequisites:**
- Your app running at `http://localhost:3000` (or update `BASE_URL` below)
- API keys set in `.env`

In [1]:
import sys
import asyncio
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(Path("packages/demo-notebook/.env"))


# Point to the repo root (two levels up from packages/demo-notebook)
REPO_ROOT = Path("../..")
BASE_URL = "http://localhost:3000"

print(f"Repo root: {REPO_ROOT.resolve()}")
print(f"App URL:   {BASE_URL}")

Repo root: /Users/macbookpro/Documents/projects/aura
App URL:   http://localhost:3000


## 1. Analyze the git diff

See what UI files changed and what task would be auto-generated.

In [2]:
import sys
sys.path.insert(0, str((REPO_ROOT / "packages/demo-recorder/src").resolve()))

from demo_recorder.diff_analyzer import DiffAnalyzer

analyzer = DiffAnalyzer(REPO_ROOT.resolve())
changes, auto_task = analyzer.analyze(base="HEAD~1")

print(f"UI files changed: {len(changes)}")
for c in changes:
    print(f"  - {c.file_path} ({c.change_type}) route={c.route} interactions={c.interactions}")

print(f"\nAuto-generated task:\n{auto_task}")

UI files changed: 1
  - apps/web/app/page.tsx (modified) route=/ interactions=['click_button']

Auto-generated task:
Navigate to http://localhost:3000. Find and click any visible buttons. Take your time to show each interaction clearly. Wait 2 seconds between actions for visibility.


## 2. Run a demo recording

Launches a browser, runs the AI agent, and saves a `.webm` video.

In [ ]:
from demo_recorder.recorder import record_demo

# Prompt from the main agent describing what changed and what to verify.
# When set, this is fused with the git diff context to build a smarter browser task.
# Example (what Claude Code would pass):
# PROMPT = "I redesigned the homepage hero. Test that the heading is visible, the Start Demo Recording button works, and the feature cards are displayed."
PROMPT = None  # set to a string to override with a targeted prompt

# Set to a string to fully override the task (ignores git diff + prompt)
CUSTOM_TASK = None
MAX_STEPS = 10
MODEL = "browser-use"  # or "gpt-4o", "claude-3-5-sonnet-latest", "gemini-2.0-flash"

output_path = await record_demo(
    repo_path=str(REPO_ROOT.resolve()),
    base_url=BASE_URL,
    task=CUSTOM_TASK,
    prompt=PROMPT,
    max_steps=MAX_STEPS,
    model=MODEL,
)

print(f"
Saved to: {output_path}")
for f in sorted(output_path.iterdir()):
    print(f"  {f.name}")

## 3. Run with a custom task

Override the task with whatever you want the agent to do.

In [ ]:
custom_task = """
Navigate to http://localhost:3000.
Scroll down to see the full page.
Click on any buttons or links you find.
Wait 2 seconds between each action.
"""

output_path = await record_demo(
    repo_path=str(REPO_ROOT.resolve()),
    base_url=BASE_URL,
    task=custom_task,
    max_steps=8,
    model="browser-use",
)

print(f"Saved to: {output_path}")